In [46]:
#!/usr/bin/env python
import json, openai,requests
import warnings
import os
warnings.filterwarnings('ignore')
import configparser
import datetime
from time import sleep

In [47]:
readKey = configparser.ConfigParser()
readKey.read_file(open('apidata.config'))

openai_org = readKey['OPENAI']["ORG"]
openai_key = readKey['OPENAI']['KEY'] 

In [48]:
import tiktoken
import pandas as pd
def num_tokens_from_string(string: str, 
                           encoding_name: str = 'text-davinci-003') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [49]:
num_tokens_from_string("This is my super prompt from Insight Builder")

8

In [50]:
get_role = input("What role you want me to play: ")

What role you want me to play: Data Summarizer


In [51]:
#reading in the dataset sample_data.csv

with open('space_titanic.csv','r') as sd:
    data = sd.readlines()

In [54]:
data[0]

'PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n'

In [59]:
data_chunks = []
rng1 = 6

In [60]:
data[6:11]

['0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\n',
 '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True\n',
 '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True\n',
 '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True\n',
 '0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True\n']

In [8]:
##Version the prompt listing questions

In [61]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)

prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and list 5 statistical, factual and inferential questions    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 6 and 11
Number of tokens for the dataset is 376

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True\n', '0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True\n', '0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True\n', '0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True\n', '0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True\n']

Prompt:Refer to the [dataset] and list 5 statistical, factual and inferential questions    
Answer:



In [10]:
def chunk_to_df(chunk_num,data_chunk):
    
    start = data_chunk.split('\\n')
    
    clean_step1 = start[0].strip(' \n').replace('=','').replace(f'dataset_{chunk_num}','')

    clean_step2 = clean_step1.split('\n+[')
    
    header = clean_step2[0]
    row1 = clean_step2[1]
    
    clean_row = []
    for dat in start[1:]:
        clean_row.append(dat.strip(" ',"))
        
    clean_row.insert(0,row1.strip("'"))
    clean_row.insert(0,header)
    
    clean_data_list = [data.split(',') for data in clean_row]
    
    chunk_df = pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])
    
    return chunk_df

In [57]:
def extract_summary(org,key,
                    role_player,
                    request_content,
                   file_name='summary_tutorial.txt'):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}."}
            ]
        )
    with open(file_name,'a+') as sumry:
        sumry.write(f'Completion for \n {request_content}')
        sumry.write('******Completion starts****** \n')
        sumry.write(str(response['choices'][0]['message']['content']))
        sumry.write('*****Completion ends****** \n')
    return response['choices'][0]['message']['content']

In [13]:
inference_1 = extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [63]:
print(inference_1)

Statistical Questions:
1. What is the mean age of the passengers in the dataset?
2. What is the median number of days the passengers spent in cryosleep?
3. What is the standard deviation of the VIP status of the passengers?
4. What is the correlation between the age of the passengers and the amount spent on shopping mall services?
5. What is the mode of the cabin types used by the passengers?

Factual Questions:
1. How many passengers were transported in total?
2. What was the most commonly visited destination by the passengers?
3. How many passengers used the Room Service during their trip?
4. What was the average age of the passengers who visited the Spa?
5. How many passengers came from Earth?

Inferential Questions:
1. Can we infer that the passengers who used VRDeck were more likely to use Spa as well?
2. Can we infer that there is a relationship between age and VIP status based on the data?
3. Can we infer that passengers who had higher expenditures on Shopping Mall services were

In [64]:
questions_list = inference_1.split('\n')

In [65]:
questions_list

['Statistical Questions:',
 '1. What is the average age of passengers on this trip?',
 '2. What is the standard deviation of the shopping mall expenses for the passengers?',
 '3. What is the correlation between VIP status and room service expenses among passengers? ',
 '4. What is the mean food court expense of passengers transported from Europa? ',
 '5. What is the proportion of passengers who used the VRDeck facility?',
 '',
 'Factual Questions:',
 '1. What is the name of the passenger transported on ship 0001_01? ',
 '2. Which passenger had the highest spa expenses?',
 '3. What is the cabin type of the passenger named Juanna Vines? ',
 '4. What was the destination of the passenger transported on ship 0003_02? ',
 '5. What is the home planet of the passenger with the lowest room service expenses? ',
 '',
 'Inferential Questions:',
 '1. Can we infer that younger passengers on this trip spent less on room service and spa facilities?',
 '2. Is there evidence to suggest that passengers w

In [66]:
questions_list[1:6]

['1. What is the average age of passengers on this trip?',
 '2. What is the standard deviation of the shopping mall expenses for the passengers?',
 '3. What is the correlation between VIP status and room service expenses among passengers? ',
 '4. What is the mean food court expense of passengers transported from Europa? ',
 '5. What is the proportion of passengers who used the VRDeck facility?']

In [67]:
questions_list[8:13]

['1. What is the name of the passenger transported on ship 0001_01? ',
 '2. Which passenger had the highest spa expenses?',
 '3. What is the cabin type of the passenger named Juanna Vines? ',
 '4. What was the destination of the passenger transported on ship 0003_02? ',
 '5. What is the home planet of the passenger with the lowest room service expenses? ']

In [68]:
questions_list[15:21]

['1. Can we infer that younger passengers on this trip spent less on room service and spa facilities?',
 '2. Is there evidence to suggest that passengers with VIP status spent more on shopping mall and food court expenses?',
 '3. Can we infer that passengers transported from Earth spent more on VRDeck usage than those from Europa?',
 '4. Is there a significant difference in cabin types between passengers with and without VIP status?',
 "5. Can we infer that the amount spent on spa, VRDeck, and shopping mall facilities are predictors of a passenger's home planet?"]

In [19]:
questions_list[8:14]

['1. What is the name of the passenger transported on ship 0001_01? ',
 '2. Which passenger had the highest spa expenses?',
 '3. What is the cabin type of the passenger named Juanna Vines? ',
 '4. What was the destination of the passenger transported on ship 0003_02? ',
 '5. What is the home planet of the passenger with the lowest room service expenses? ',
 '']

In [ ]:
questions_list = ['Tell me about the number of rows in dataset','Who are the travellers in the dataset?']

In [ ]:
questions_list = [questions_list[0],questions_list[10],questions_list[15:20]]

In [72]:
type(questions_list[8:13])

list

In [69]:
quest_string = ''
for string in questions_list[8:13]:
    quest_string = quest_string + string.split('.')[1]

In [70]:
quest_string

' What is the name of the passenger transported on ship 0001_01?  Which passenger had the highest spa expenses? What is the cabin type of the passenger named Juanna Vines?  What was the destination of the passenger transported on ship 0003_02?  What is the home planet of the passenger with the lowest room service expenses? '

In [25]:
###Follow up design prompt

In [73]:
rng1 = 1
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and answer succinctly to the following questions. 
{quest_string}
At the end print COMPLETED designed INFERENCE

Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 1 and 6
Number of tokens for the dataset is 460

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\n', '0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\n']

Prompt:Refer to the [dataset] and answer succinctly to the following questions. 
 What is the name of the passenger transported on ship 0001_01?  Which passenger had the highest spa expenses? What is the cabin type of the passenger named Juanna Vines?  What was the destination of the passeng

In [35]:
designed_summary =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [36]:
print(designed_summary)

The name of the passenger transported on ship 0001_01 is Maham Ofracculy. The passenger with the highest spa expenses is Altark Susent. The cabin type of the passenger named Juanna Vines is F/0/S. The destination of the passenger transported on ship 0003_02 is TRAPPIST-1e. The home planet of the passenger with the lowest room service expenses is Europa. COMPLETED designed INFERENCE.


In [37]:
num_tokens_from_string(designed_summary)

93

In [ ]:
#check if gpt can make such inferences directly

In [77]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and succinctly list 10 statistical, factual and inferences    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 41 and 46
Number of tokens for the dataset is 377

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0039_01,Earth,True,G/1/P,55 Cancri e,30.0,False,0.0,0.0,,0.0,0.0,Jorgie Batthewitt,False\n', '0041_01,Earth,True,G/2/P,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Margia Moodsey,True\n', '0043_01,Europa,False,B/3/P,TRAPPIST-1e,45.0,False,0.0,164.0,45.0,2511.0,855.0,Ankalik Cylistrand,False\n', '0044_01,Earth,True,G/3/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Jodye Coopelandez,False\n', '0044_02,Earth,True,G/3/P,55 Cancri e,4.0,False,0.0,0.0,0.0,0.0,0.0,Kayne Coopelandez,True\n']

Prompt:Refer to the [dataset] and succinctly list 10 statistical, factual and inferences    
Answer:



In [78]:
inference_possiblity =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [79]:
inference_possiblity

'1. The dataset contains information on individuals who have undergone cryosleep.\n2. Home planet, cabin, destination, and age are recorded for each individual.\n3. There are VIP individuals who have access to room service, food court, shopping mall, spa, and VR deck.\n4. Some individuals have been transported to their destination while some have not.\n5. The dataset contains information on individuals from multiple planets, including Earth and Europa.\n6. The age of individuals ranges from 4 to 55 years old.\n7. There are individuals who were transported to both TRAPPIST-1e and 55 Cancri e.\n8. The dataset contains at least 5 entries.\n9. There are individuals with similar last names (Coopelandez).\n10. There is missing data for some amenities, as some individuals have NaN values for shopping mall or spa access.'

### Creating a loop for completing the entire dataset

In [74]:
for x in range(1,5):
    print(x)

1
2
3
4


In [78]:
8600/25

344.0

In [83]:
inference_requests = []
rng1 = 1
for x in range(1,5):
    rng2 = rng1 + 25
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    #print(data_t)
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and in 300 words answer the following questions.
    Do not print the questions. 
    {quest_string}
    At the end print COMPLETED INFERENCE

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    inference_requests.append(prompt_made)
    rng1 = rng1 + 25

before spliting data 1 and 26
Number of tokens for the dataset is 1636
before spliting data 26 and 51
Number of tokens for the dataset is 1625
before spliting data 51 and 76
Number of tokens for the dataset is 1635
before spliting data 76 and 101
Number of tokens for the dataset is 1630


In [84]:
print(inference_requests[3])


    dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0082_01,Mars,False,F/16/P,TRAPPIST-1e,42.0,False,7406.0,0.0,0.0,0.0,0.0,Totse Datte,False\n', '0082_02,Mars,True,F/16/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Eaturs Datte,True\n', '0082_03,Mars,False,F/16/P,TRAPPIST-1e,8.0,False,0.0,0.0,0.0,0.0,0.0,,True\n', '0084_01,Earth,False,G/14/S,TRAPPIST-1e,24.0,False,688.0,0.0,0.0,0.0,17.0,Coren Coopezmaney,True\n', '0085_01,Europa,True,C/5/S,TRAPPIST-1e,40.0,False,0.0,0.0,0.0,0.0,0.0,Furudah Ellcefulve,True\n', '0086_01,Earth,False,F/17/P,TRAPPIST-1e,43.0,False,211.0,0.0,0.0,638.0,513.0,Jodye Kinson,False\n', '0088_01,Mars,True,E/5/S,PSO J318.5-22,45.0,False,0.0,0.0,0.0,0.0,0.0,Stmeal Sacre,True\n', '0090_01,Earth,True,G/15/S,TRAPPIST-1e,35.0,False,0.0,0.0,0.0,0.0,0.0,Heremy Santry,True\n', '0091_01,Earth,True,G/16/S,TRAPPIST-1e,26.0,False,,0.0,0.0,0.0,0.0,Deanne Yorkland,True\n', '0091_02,Earth,False,F/

In [85]:
len(inference_requests)

4

In [86]:
for ind,inf in enumerate(inference_requests):
    comp = extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=inf)

In [87]:
comp

'After analyzing the provided dataset, we can answer the following questions:\n\n1. What is the name of the passenger transported on ship 0001_01?\nUnfortunately, we cannot answer this question because the dataset does not include ship 0001_01.\n\n2. Which passenger had the highest spa expenses?\nThe passenger with the highest spa expenses is Jodye Kinson, transported to TRAPPIST-1e from Earth on ship 0086_01. Their spa expenses were $513.\n\n3. What is the cabin type of the passenger named Juanna Vines?\nWe cannot answer this question because the dataset does not include information about the cabin type of a passenger named Juanna Vines.\n\n4. What was the destination of the passenger transported on ship 0003_02?\nWe cannot answer this question because the dataset does not include information about the destination of a passenger transported on ship 0003_02.\n\n5. What is the home planet of the passenger with the lowest room service expenses?\nThe passenger with the lowest room service

### Can we AI to remember the dataset and finetune the prompt???